# Custom models

#### Custom model

In [1]:
import tensorflow as tf

In [2]:
input_a=tf.keras.layers.Input(shape=[1])
input_b=tf.keras.layers.Input(shape=[1])

hidden_1=tf.keras.layers.Dense(32,activation='relu')(input_a)
hidden_2=tf.keras.layers.Dense(32,activation='relu')(hidden_1)

output=tf.keras.layers.Dense(1)(hidden_2)

concat=tf.keras.layers.concatenate([hidden_2,input_b])
output2=tf.keras.layers.Dense(1)(concat)

model=tf.keras.Model(inputs=[input_a,input_b],outputs=[output,output2])

#### Model as a class

In [9]:
class MyModel(tf.keras.models.Model):
    def __init__(self,units=32,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.hidden1=tf.keras.layers.Dense(units,activation=activation)
        self.hidden2=tf.keras.layers.Dense(units,activation=activation)
        self.output1=tf.keras.layers.Dense(1)
        self.output2=tf.keras.layers.Dense(1)
    def call(self,inputs):
        input_a,input_b=inputs
        hidden_1=self.hidden_1(input_a)
        hidden_2=self.hidden_1(input_b)
        concat=tf.keras.layers.concatenate([hidden_2,input_b])
        output=self.output1(hidden_2)
        output2=self.output2(concat)
        return output1, output2

In [7]:
model=MyModel()

### Subclassing models

- extends functionality of models class
- functional & sequential code
- modular architecture

## Simplify complex architectures

`ResNet` - residual network - network with "scrouts" over some layers so that the network doesn't loose the info about the data too quickly. They are less prune to having the increasing error together with the bigger depth of the network.

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds

C:\Users\pk764\miniconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Conv NN Res

In [3]:
class CNNResidual(tf.keras.layers.Layer):
    def __init__(self, layers, filters, **kwargs):
        super().__init__(**kwargs)
        self.hidden=[tf.keras.layers.Conv2D(filters,(3,3),activation='relu') for _ in range(layers)]
    
    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return inputs + x

#### Deep NN Res

In [4]:
class DNNResidual(tf.keras.layers.Layer):
    def __init__(self, layers, neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden=[tf.keras.layers.Dense(neurons,(3,3),activation='relu') for _ in range(layers)]
    
    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return inputs + x

### Final network with bloks of CNNRes and DNNRes

In [6]:
class MyRes(tf.keras.models.Model):
    def __init__(self, **kwargs):
        self.hidden1 = tf.keras.layers.Dense(30,activation='relu')
        self.block1 = CNNResidual(2,32)
        self.block2 = DNNResidual(2,64)
        self.out = tf.keras.layers.Dense(1)
    def call(self, inputs):
        x = self.hidden1(inpiuts)
        x = self.block1(x)
        for _ in range(1,4):
            x = self.block2(x)
        return self.out(x)

---

### ResNet

ResNet consists of several IdentityBlock elements

In [3]:
class IdentityBlock(tf.keras.models.Model):
    def __init__(self,filters,kernel_size):
        super(IdentityBlock,self).__init__(name='')
        
        self.conv1=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.bn1=tf.keras.layers.BatchNormalization()
        self.conv2=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
        self.bn2=tf.keras.layers.BatchNormalization()
        self.act=tf.keras.layers.Activation('relu')
        self.add=tf.keras.layers.Add()

    def call(self,inputs):
        x=self.conv1(inputs)
        x=self.bn1(x)
        x=self.act(x)
        
        x=self.conv2(inputs)
        x=self.bn2(x)
        x=self.act(x)

        x=self.add([x,inputs])
        x=self.act(x)
        
        return x

In [4]:
class ResNet(tf.keras.models.Model):
    def __init__(self,num_classes):
        super(ResNet,self).__init__()
        
        self.conv=tf.keras.layers.Conv2D(64,7,padding='same')
        self.bn=tf.keras.layers.BatchNormalization()
        self.act=tf.keras.layers.Activation('relu')
        self.max_pool=tf.keras.layers.MaxPool2D((3,3))

        self.idla=IdentityBlock(64,3)
        self.idlb=IdentityBlock(64,3)

        self.global_pool=tf.keras.layers.GlobalAveragePooling2D()
        self.classifier=tf.keras.layers.Dense(num_classes,activation='softmax')
        
    def call(self,inputs):
        x=self.conv(inputs)
        x=self.bn(x)
        x=self.act(x)
        x=self.max_pool(x)

        x=self.idla(x)
        x=self.idlb(x)

        x=self.global_pool(x)
        x=self.classifier(x)

        return x

In [5]:
def preprocess(data):
    return tf.cast(data,['image'],tf.float32/255,data['label'])

In [7]:
# dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
# dataset = dataset.map(preprocess).batch(32)

In [8]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

In [10]:
model=ResNet(10)
model.compile(loss='sparce_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=1)

### Dynamic variables:

In [1]:
class MyClass():
    def __init__(self):
        self.var1=0

In [2]:
my_obj=MyClass()

In [3]:
my_obj.__dict__

{'var1': 0}

In [4]:
vars(my_obj)

{'var1': 0}

In [5]:
my_obj.var2=2
vars(my_obj)

{'var1': 0, 'var2': 2}

In [6]:
vars(my_obj)['var3']=3
vars(my_obj)

{'var1': 0, 'var2': 2, 'var3': 3}

In [7]:
for i in range(3,7):
    vars(my_obj)[f'var{i}']=i
vars(my_obj)

{'var1': 0, 'var2': 2, 'var3': 3, 'var4': 4, 'var5': 5, 'var6': 6}